In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df =pd.read_csv('/kaggle/input/water-quality-prediction/Water Quality Prediction.csv')

In [ ]:
df

In [ ]:
data =df.drop(['Month', 'Day', 'Time of Day'], axis=1)

In [ ]:
data.head(5)

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

In [ ]:
data['Color'].value_counts()

In [ ]:
data.drop('Source', axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Create a LabelEncoder object
le = LabelEncoder()

# Fit the LabelEncoder object to the categorical data
le.fit(data['Color'])

# Transform the categorical data using the LabelEncoder object
data['Color'] = le.transform(data['Color'])

In [ ]:
X= data.drop(['Target', 'Source'], axis=1)
y = data.Target

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# we will divide the dataset into training and testing dataset
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=99)

In [ ]:
# check the shape again
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
X_train

In [ ]:
# scaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [ ]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
# import the libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv1D,BatchNormalization,Dropout

In [ ]:
# import model
model=Sequential()
# layers
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# build ANN
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))

In [ ]:
# summary
model.summary()

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
# fitting the model
history=model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

In [ ]:
# plot
import matplotlib.pyplot as plt
def plot_learningcurve(history,epochs):
  epoch=range(1,epochs+1)
  # accuracyplot_learningcurve(history,20)
  plt.plot(epoch, history.history['accuracy'])
  plt.plot(epoch, history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.legend(['train','val'], loc='upper left')
  plt.show()

  # loss
  plt.plot(epoch, history.history['loss'])
  plt.plot(epoch, history.history['val_loss'])
  plt.title('Model loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend(['train','val'], loc='upper left')
  plt.show()

In [ ]:
plot_learningcurve(history,20)

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred= np.round(y_pred_prob)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_test.shape

In [ ]:

from sklearn.metrics import confusion_matrix
import seaborn as sns
 
plt.figure(figsize=(16,9))
y_pred_labels = [ np.argmax(label) for label in y_pred ]
cm = confusion_matrix(y_test, y_pred_labels)
 
# show cm 
sns.heatmap(cm, annot=True, fmt='d',xticklabels=y_pred_labels, yticklabels=y_pred_labels)
 
from sklearn.metrics import classification_report
cr= classification_report(y_test, y_pred_labels, target_names=class_labels)
print(cr)

In [ ]:
y_pred_labels = [ np.argmax(label) for label in y_pred ]
conf_mat = confusion_matrix(y_test,y_pred_labels)
sns.set_style(style='dark')
plt.figure(figsize=(12,8))
heatmap = sns.heatmap(conf_mat,vmin=np.min(conf_mat.all()), vmax=np.max(conf_mat), annot=True,fmt='d', annot_kws={"fontsize":20},cmap='Spectral')
heatmap.set_title('Confusion Matrix', fontdict={'fontsize':15}, pad=12)
heatmap.set_xlabel('Predicted',fontdict={'fontsize':14})
heatmap.set_ylabel('Actual',fontdict={'fontsize':14})
heatmap.set_xticklabels(['NO','YES'], fontdict={'fontsize':12})
heatmap.set_yticklabels(['NO','YES'], fontdict={'fontsize':12})
plt.show()




In [ ]:
from sklearn.metrics import classification_report

# Print the classification report
print(classification_report(y_test,y_pred_labels))